<a href="https://colab.research.google.com/github/DoubekSeth/CS5350HW/blob/main/DecisionTree/CS5350HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and data sets

## Libraries

In [1036]:
from collections import Counter
import queue
import math
import statistics
import copy
import numpy as np
import pandas as pd

## Datasets

### Tennis

In [1037]:
s_tennis = [["sunny", "hot", "high", "weak", "-"],
     ["sunny", "hot", "high", "strong", "-"],
     ["overcast", "hot", "high", "weak", "+"],
     ["rainy", "mild", "high", "weak", "+"],
     ["rainy", "cool", "normal", "weak", "+"],
     ["rainy", "cool", "normal", "strong", "-"],
     ["overcast", "cool", "normal", "strong", "+"],
     ["sunny", "mild", "high", "weak", "-"],
     ["sunny", "cool", "normal", "weak", "+"],
     ["rainy", "mild", "normal", "weak", "+"],
     ["sunny", "mild", "normal", "strong", "+"],
     ["overcast", "mild", "high", "strong", "+"],
     ["overcast", "hot", "normal", "weak", "+"],
     ["rainy", "mild", "high", "strong", "-"]]
full_attributes_tennis = {"outlook":["sunny", "overcast", "rainy"], "temperature":["hot", "mild", "cool"], "humidity":["high", "normal", "low"], "wind":["strong", "weak"]}
remaining_attributes_tennis = ["outlook", "temperature", "humidity", "wind"]

### Boolean Table Example (1st problem)


In [1038]:
s_booltab = [
    ["0", "0", "1", "0", "0"],
    ["0", "1", "0", "0", "0"],
    ["0", "0", "1", "1", "1"],
    ["1", "0", "0", "1", "1"],
    ["0", "1", "1", "0", "0"],
    ["1", "1", "0", "0", "0"],
    ["0", "1", "0", "1", "0"]
]
full_attributes_booltab = {"x1":["0", "1"], "x2":["0", "1"], "x3":["0", "1"], "x4":["0", "1"]}
remaining_attributes_booltab = ["x1", "x2", "x3", "x4"]

### Car

In [1039]:
s_cartraining = pd.read_csv("https://raw.githubusercontent.com/DoubekSeth/ToyDatasets/main/car-4/train.csv", header=None).to_numpy()
s_cartesting = pd.read_csv("https://raw.githubusercontent.com/DoubekSeth/ToyDatasets/main/car-4/test.csv", header=None).to_numpy()

full_attributes_car = {
    "buying":["vhigh", "high", "med", "low"],
    "maint":["vhigh", "high", "med", "low"],
    "doors":["2", "3", "4", "5more"],
    "persons":["2", "4", "more"],
    "lug_boot":["small", "med", "big"],
    "safety":["low", "med", "high"]}
remaining_attributes_car = ["buying", "maint", "doors", "persons", "lug_boot", "safety"]

### Bank

In [1040]:
s_banktraining = pd.read_csv("https://raw.githubusercontent.com/DoubekSeth/ToyDatasets/main/bank-4/train.csv", header=None).to_numpy()
s_banktesting = pd.read_csv("https://raw.githubusercontent.com/DoubekSeth/ToyDatasets/main/bank-4/test.csv", header=None).to_numpy()

full_attributes_bank = {
    "age":"numeric",
    "job":["admin.","unknown","unemployed","management","housemaid","entrepreneur","student", "blue-collar","self-employed","retired","technician","services"],
    "marital":["married","divorced","single"],
    "education":["unknown","secondary","primary","tertiary"],
    "default":["yes", "no"],
    "balance":"numeric",
    "housing":["yes", "no"],
    "loan":["yes", "no"],
    "contact":["unknown","telephone","cellular"],
    "day":"numeric",
    "month":["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"],
    "duration":"numeric",
    "campaign":"numeric",
    "pdays":"numeric",
    "previous":"numeric",
    "poutcome":["unknown","other","failure","success"]
}
remaining_attributes_bank = ["age", "job", "marital", "education", "default", "balance", "housing", "loan", "contact", "day", "month", "duration", "campaign", "pdays", "previous", "poutcome"]

Filling in unknowns in dataset

In [1041]:
#Find all modes for each data point
attributes_mode_bank = {}
for i in range(len(remaining_attributes_bank)):
  mode = max(set([k for k in s_banktraining[:, i] if k != "unknown"]), key=[k for k in s_banktraining[:, i] if k != "unknown"].count)
  attributes_mode_bank[remaining_attributes_bank[i]] = mode
print(attributes_mode_bank)

# Set the data back
s_banktraining_filled = copy.deepcopy(s_banktraining)
for datapoint in range(len(s_banktraining_filled)):
  for eleInd in range(len(s_banktraining_filled[datapoint])):
    if(s_banktraining_filled[datapoint, eleInd]=="unknown"):
      s_banktraining_filled[datapoint, eleInd]=attributes_mode_bank[remaining_attributes_bank[eleInd]]
print(s_banktraining_filled)

# Set the data for testing back
s_banktesting_filled = copy.deepcopy(s_banktesting)
for datapoint in range(len(s_banktesting_filled)):
  for eleInd in range(len(s_banktesting_filled[datapoint])):
    if(s_banktesting_filled[datapoint, eleInd]=="unknown"):
      s_banktesting_filled[datapoint, eleInd]=attributes_mode_bank[remaining_attributes_bank[eleInd]]

full_attributes_bank_filled = {
    "age":"numeric",
    "job":["admin.","unemployed","management","housemaid","entrepreneur","student", "blue-collar","self-employed","retired","technician","services"],
    "marital":["married","divorced","single"],
    "education":["secondary","primary","tertiary"],
    "default":["yes", "no"],
    "balance":"numeric",
    "housing":["yes", "no"],
    "loan":["yes", "no"],
    "contact":["telephone","cellular"],
    "day":"numeric",
    "month":["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"],
    "duration":"numeric",
    "campaign":"numeric",
    "pdays":"numeric",
    "previous":"numeric",
    "poutcome":["other","failure","success"]
}

{'age': 32, 'job': 'blue-collar', 'marital': 'married', 'education': 'secondary', 'default': 'no', 'balance': 0, 'housing': 'yes', 'loan': 'no', 'contact': 'cellular', 'day': 20, 'month': 'may', 'duration': 85, 'campaign': 1, 'pdays': -1, 'previous': 0, 'poutcome': 'failure'}
[[41 'services' 'married' ... 0 'failure' 'no']
 [48 'blue-collar' 'single' ... 0 'failure' 'no']
 [55 'technician' 'married' ... 3 'success' 'yes']
 ...
 [37 'blue-collar' 'married' ... 0 'failure' 'no']
 [31 'management' 'single' ... 0 'failure' 'no']
 [28 'blue-collar' 'married' ... 0 'failure' 'no']]


# ID3 Algorithm

## Main Algorithm

In [1042]:
class Node:
  def __init__(self, s, atrs, parent=None, parentVal=None, label=None):
    self.s = s
    self.atrs = atrs
    self.parent = parent
    self.label = label
    self.children = []
    self.parentVal = parentVal

  def addChild(self, node):
    self.children.append(node)

In [1043]:
def ID3(s, remaining_attributes, full_attributes, purity_func, parentNode, weights=None, level=0, stopping=math.inf, printing=False):
  if weights is None:
    weights=np.ones(len(s))
  #First, check if all training examples s have same label. First stopping condition
  if(labelSame(s)[0]):
    if(printing):
      print("Same Label", labelSame(s)[1])
    parentNode.label = labelSame(s)[1]
    return
  #Second, check if exhausted attributes and if so return leaf node with most common label. Second stopping condition
  if(len(remaining_attributes) == 0):
    if(printing):
      print("Exhausted Attributes", mostCommonLabel(s, 1)[0][0])
    parentNode.label = mostCommonLabel(s, 1)[0][0]
    return
  #Third, check if have hit maximum stopping depth
  #print("hit ", level, " out of ", stopping)
  if(level >= stopping):
    parentNode.label = mostCommonLabel(s, 1)[0][0]
    return
  level += 1
  #If didn't stop, then we can move onto the main part of the algorithm
  #Create root if we don't have one
  if(parentNode==None):
    parentNode = Node(s=s, atrs=remaining_attributes, parent=None)
  #Find attribute best splits S
  bestAtr = findBestSplit(s, weights, remaining_attributes, full_attributes, purity_func, printing)
  parentNode.label = bestAtr
  if(printing):
    print(bestAtr)
  less_remaining_attributes = remaining_attributes[:]
  #print(less_remaining_attributes, bestAtr)
  less_remaining_attributes.remove(bestAtr)
  #Go thru all values for best attribute
  loop_attrs = full_attributes[bestAtr]
  numericFilter=False
  if(loop_attrs=="numeric"):
    numericFilter=True
    loop_attrs = ["lower than median", "higher than median"]
  for val in loop_attrs:
    s_v, weights_v = filterSOnAtrVal(s, weights, bestAtr, val, full_attributes, numericFilter)
    if(printing):
      print("Length check", len(s_v), len(weights_v))
    #Check if s_v empty
    if(len(s_v)==0):
      if(printing):
        print("S_v empty", val, mostCommonLabel(s, 1)[0][0])
      newNode = Node(s=s_v, atrs=less_remaining_attributes, parent=parentNode, parentVal=val, label=mostCommonLabel(s, 1)[0][0])
      parentNode.addChild(newNode)
    else:
      newNode = Node(s=s_v, atrs=less_remaining_attributes, parent=parentNode, parentVal=val)
      if(printing):
        print("New node", val)
      parentNode.addChild(newNode)
      ID3(s_v, less_remaining_attributes, full_attributes, purity_func, newNode, weights=weights_v, level=level, stopping=stopping, printing=printing)
  return parentNode

## Helper Functions

Does two functions, first returns true if all labels are the same for s. Second returns the label if all labels are same or else returns none

In [1044]:
def labelSame(s):
  label = s[0][len(s[0])-1]
  for i in s:
    if(i[len(i)-1] != label):
      return False, None
  return True, label

Another helper function, returns the n most common label in s

In [1045]:
def mostCommonLabel(s, n):
  labels = []
  for i in s:
    labels.append(i[len(i)-1])
  #Find most common label out of all the labels
  mostCommon = Counter(labels).most_common(n)
  return mostCommon

This helper finds the attribute that results in the best split

In [1046]:
def findBestSplit(s, weights, atrs, full_attributes, purity_func, printing):
  max_info_gain = 0
  max_info_gain_atr = atrs[0]
  for atr in atrs:
    info_gained = find_info_gained(s, weights, atr, full_attributes, purity_func, printing)
    if(printing):
      print(atr, info_gained)
    if(info_gained > max_info_gain):
      max_info_gain = info_gained
      max_info_gain_atr = atr
  return max_info_gain_atr

This helper gives the information gained from a single attribute

In [1047]:
def find_info_gained(s, weights, atr, full_attributes, purity_func, printing):
  if(full_attributes[atr]=="numeric"):
    return find_numeric_info_gained(s, weights, atr, full_attributes, purity_func, printing)
  else:
    return find_categorical_info_gained(s, weights, atr, full_attributes, purity_func, printing)

In [1048]:
def find_numeric_info_gained(s, weights, atr, full_attributes, purity_func, printing):
  if(printing):
    print(atr)
  atr_ind = list(full_attributes.keys()).index(atr)
  #Create dictionary that contains attribute categories and what labels each attribute category has
  atr_type_count_dict = {"low":[], "high":[]}
  count = 0
  median = statistics.median(x[atr_ind] for x in s)
  for i in s:
    if i[atr_ind] < median:
      atr_type_count_dict["low"].append([i[len(i)-1], weights[count]])
    else:
      atr_type_count_dict["high"].append([i[len(i)-1], weights[count]])
    count += 1
  #This puts s into format of (label, weight)
  pur_s = []
  count = 0
  for i in s:
    pur_s.append([i[len(i)-1], weights[count]])
    count += 1
  starting_info = purity_func(pur_s)
  if(printing):
    print("Starting Info: ", starting_info)
  weighted_sum = 0
  for j in atr_type_count_dict.values():
    if(printing):
      print("J", j)
    weighted_sum += np.sum(np.fromiter((i[1] for i in j), dtype=float))/np.sum(weights)*purity_func(j)
  return starting_info-weighted_sum

In [1049]:
def find_categorical_info_gained(s, weights, atr, full_attributes, purity_func, printing):
  atr_ind = list(full_attributes.keys()).index(atr)
  if(printing):
    print(atr)
  #Create dictionary that contains attribute categories and what labels each attribute category has
  atr_type_count_dict = {}
  count = 0
  for i in s:
    if i[atr_ind] in atr_type_count_dict:
      atr_type_count_dict[i[atr_ind]].append([i[len(i)-1], weights[count]])
    else:
      atr_type_count_dict[i[atr_ind]] = [[i[len(i)-1], weights[count]]]
    count += 1
  #Use proportions to run info gain function and find total information gain
  #This puts s into format of (label, weight)
  pur_s = []
  count = 0
  for i in s:
    pur_s.append([i[len(i)-1], weights[count]])
    count += 1
  starting_info = purity_func(pur_s)
  if(printing):
    print("Starting Info: ", starting_info)
  weighted_sum = 0
  for j in atr_type_count_dict.values():
    if(printing):
      print("J", j)
    weighted_sum += np.sum(np.fromiter((i[1] for i in j), dtype=float))/np.sum(weights)*purity_func(j)
  return starting_info-weighted_sum

Filters S based on an attribute value to get S_v

In [1050]:
def filterSOnAtrVal(s, weights, atr, val, full_attributes, numericFilter):
  filtered = []
  filteredWeights = []
  count = 0
  atr_ind = list(full_attributes.keys()).index(atr)
  for i in s:
    if(numericFilter):
      if(val == "lower than median"):
        if(i[atr_ind] < statistics.median(x[atr_ind] for x in s)):
          filtered.append(i)
          filteredWeights.append(weights[count])
      else:
        if(i[atr_ind] >= statistics.median(x[atr_ind] for x in s)):
          filtered.append(i)
          filteredWeights.append(weights[count])
    else:
      if(i[atr_ind] == val):
        filtered.append(i)
        filteredWeights.append(weights[count])
    count += 1
  return filtered, filteredWeights

## Purity Functions

In [1051]:
def entropy(labels):
  labeldf = pd.DataFrame(labels, columns=["Label", "Weight"])
  grouped = labeldf.groupby("Label").sum()
  vec_counts = np.array(grouped).flatten()
  vec_props = vec_counts/np.sum(vec_counts)
  return -np.dot(vec_props, np.log2(vec_props))

In [1052]:
def GiniInd(labels):
  labeldf = pd.DataFrame(labels, columns=["Label", "Weight"])
  grouped = labeldf.groupby("Label").sum()
  vec_counts = np.array(grouped).flatten()
  vec_props = vec_counts/np.sum(vec_counts)
  return 1-np.dot(vec_props, vec_props)

In [1053]:
def majorityError(labels):
  labeldf = pd.DataFrame(labels, columns=["Label", "Weight"])
  grouped = labeldf.groupby("Label").sum()
  vec_counts = np.array(grouped).flatten()
  vec_props = vec_counts/np.sum(vec_counts)
  vec_props.sort()
  return np.sum(vec_props[:len(vec_props)-1])

## Display Tree

In [1054]:
def unravelTree(q1, q2, childQ, level):
  print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
  while(not q1.empty()):
    node = q1.get()
    childThere = childQ.get()
    firstChild = True
    print("ParentVal: ", node.parentVal)
    print("Label: ", node.label)
    if(not childThere):
      print("----")
    for child in node.children:
      q2.put(child)
      if(firstChild):
        firstChild = False
      else:
        childQ.put(True)
    if(len(node.children)!= 0):
      childQ.put(False)
  level += 1
  q1 = q2
  q2 = queue.Queue()
  if(q1.empty()):
    return
  unravelTree(q1, q2, childQ, level)

# Run ID3

## Set Data

In [1055]:
index = 3
s = [s_booltab, s_tennis, s_cartraining, s_banktraining, s_banktraining_filled][index]
s_test = ["","", s_cartesting, s_banktesting, s_banktraining_filled][index]
rem = [remaining_attributes_booltab, remaining_attributes_tennis, remaining_attributes_car, remaining_attributes_bank, remaining_attributes_bank][index]
full_attr = [full_attributes_booltab, full_attributes_tennis, full_attributes_car, full_attributes_bank, full_attributes_bank_filled][index]
pur = [entropy, GiniInd, majorityError][0]
stop = 1

## Display Tree

In [1056]:
weights = np.ones(len(s))
weights[0] = 500
print(weights)
root = ID3(s, rem, full_attr, pur, None, weights=weights, stopping=stop, printing=True)

q = queue.Queue()
q2 = queue.Queue()
q.put(root)
childQ = queue.Queue()
childQ.put(True)


unravelTree(q, q2, childQ, 0)

[500.   1.   1. ...   1.   1.   1.]
age
Starting Info:  0.49502029027646954
J [['yes', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['yes', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['yes', 1.0], ['no', 1.0], ['no', 1.0], ['yes', 1.0], ['yes', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['yes', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['yes', 1.0], ['no', 1.0], ['yes', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['y

## Evaluate Model

Finds accuracy of model on test data

In [1057]:
def findAcc(root, test, full_attr):
  matched = 0
  for i in test:
    matched += travTree(root, i, full_attr)
  return matched/len(test)

Traverses the tree recursively, returns 1 if test label matches model and 0 otherwise

In [1058]:
def travTree(root, datapoint, full_attr):
  #Stopping Condition, root no children
  if(len(root.children) == 0):
    if(root.label == datapoint[len(datapoint)-1]):
      #print("matched")
      return 1
    else:
      #print("no match")
      return 0
  #Otherwise, traverse down
  curr_attr = root.label
  do_numeric = False
  if(full_attr[curr_attr] == "numeric"):
    do_numeric = True
  #print("current attribute: ", curr_attr)
  atr_ind = list(full_attr.keys()).index(curr_attr)
  for child in root.children:
    if(do_numeric):
      median = statistics.median(x[atr_ind] for x in root.s)
      #print("numeric: ", median, " ", datapoint[atr_ind])
      if((datapoint[atr_ind] < median and child.parentVal=="lower than median") or (datapoint[atr_ind] >= median and child.parentVal=="higher than median")):
        return travTree(child, datapoint, full_attr)
    else:
      if(child.parentVal == datapoint[atr_ind]):
        #print("Going down on val: ", child.parentVal)
        return travTree(child, datapoint, full_attr)

In [1059]:
root = ID3(s, rem, full_attr, pur, None, stopping=stop)

findAcc(root, s_test, full_attr)

0.8752

# Run Boolean Table

## Set Vars

In [1060]:
index = 0
s = [s_booltab, s_tennis, s_cartraining, s_banktraining, s_banktraining_filled][index]
s_test = ["","", s_cartesting, s_banktesting, s_banktraining_filled][index]
rem = [remaining_attributes_booltab, remaining_attributes_tennis, remaining_attributes_car, remaining_attributes_bank, remaining_attributes_bank][index]
full_attr = [full_attributes_booltab, full_attributes_tennis, full_attributes_car, full_attributes_bank, full_attributes_bank_filled][index]
pur = [entropy, GiniInd, majorityError][0]
stop = math.inf

## Run ID3 & Print

In [1061]:
root = ID3(s, rem, full_attr, pur, None, stopping=stop, printing=True)

q = queue.Queue()
q2 = queue.Queue()
q.put(root)
childQ = queue.Queue()
childQ.put(True)


unravelTree(q, q2, childQ, 0)

x1
Starting Info:  0.863120568566631
J [['0', 1.0], ['0', 1.0], ['1', 1.0], ['0', 1.0], ['0', 1.0]]
J [['1', 1.0], ['0', 1.0]]
x1 0.061743357932800724
x2
Starting Info:  0.863120568566631
J [['0', 1.0], ['1', 1.0], ['1', 1.0]]
J [['0', 1.0], ['0', 1.0], ['0', 1.0], ['0', 1.0]]
x2 0.46956521111470695
x3
Starting Info:  0.863120568566631
J [['0', 1.0], ['1', 1.0], ['0', 1.0]]
J [['0', 1.0], ['1', 1.0], ['0', 1.0], ['0', 1.0]]
x3 0.0059777114237739015
x4
Starting Info:  0.863120568566631
J [['0', 1.0], ['0', 1.0], ['0', 1.0], ['0', 1.0]]
J [['1', 1.0], ['1', 1.0], ['0', 1.0]]
x4 0.46956521111470695
x2
Length check 3 3
New node 0
x1
Starting Info:  0.9182958340544896
J [['0', 1.0], ['1', 1.0]]
J [['1', 1.0]]
x1 0.2516291673878229
x3
Starting Info:  0.9182958340544896
J [['0', 1.0], ['1', 1.0]]
J [['1', 1.0]]
x3 0.2516291673878229
x4
Starting Info:  0.9182958340544896
J [['0', 1.0]]
J [['1', 1.0], ['1', 1.0]]
x4 0.9182958340544896
x4
Length check 1 1
New node 0
Same Label 0
Length check 2 2

# Run Tennis Example

## Set Vars

In [1062]:
index = 1
s = [s_booltab, s_tennis, s_cartraining, s_banktraining, s_banktraining_filled][index]
s_test = ["","", s_cartesting, s_banktesting, s_banktraining_filled][index]
rem = [remaining_attributes_booltab, remaining_attributes_tennis, remaining_attributes_car, remaining_attributes_bank, remaining_attributes_bank][index]
full_attr = [full_attributes_booltab, full_attributes_tennis, full_attributes_car, full_attributes_bank, full_attributes_bank_filled][index]
pur = [entropy, GiniInd, majorityError][0]
stop = math.inf

## Run ID3

### Entropy

In [1063]:
root = ID3(s, rem, full_attr, entropy, None, stopping=stop, printing=True)

q = queue.Queue()
q2 = queue.Queue()
q.put(root)
childQ = queue.Queue()
childQ.put(True)


unravelTree(q, q2, childQ, 0)

outlook
Starting Info:  0.9402859586706311
J [['-', 1.0], ['-', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0]]
J [['+', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0]]
J [['+', 1.0], ['+', 1.0], ['-', 1.0], ['+', 1.0], ['-', 1.0]]
outlook 0.24674981977443933
temperature
Starting Info:  0.9402859586706311
J [['-', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0]]
J [['+', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0], ['-', 1.0]]
J [['+', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0]]
temperature 0.02922256565895487
humidity
Starting Info:  0.9402859586706311
J [['-', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0], ['-', 1.0], ['+', 1.0], ['-', 1.0]]
J [['+', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0]]
humidity 0.15183550136234159
wind
Starting Info:  0.9402859586706311
J [['-', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0]]
J [['-', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0], ['-', 1.0]]
wind 0.04812703040826949
outlook
Le

### Gini Index

In [1064]:
root = ID3(s, rem, full_attr, GiniInd, None, stopping=stop, printing=True)

q = queue.Queue()
q2 = queue.Queue()
q.put(root)
childQ = queue.Queue()
childQ.put(True)


unravelTree(q, q2, childQ, 0)

outlook
Starting Info:  0.4591836734693877
J [['-', 1.0], ['-', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0]]
J [['+', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0]]
J [['+', 1.0], ['+', 1.0], ['-', 1.0], ['+', 1.0], ['-', 1.0]]
outlook 0.11632653061224485
temperature
Starting Info:  0.4591836734693877
J [['-', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0]]
J [['+', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0], ['-', 1.0]]
J [['+', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0]]
temperature 0.018707482993197244
humidity
Starting Info:  0.4591836734693877
J [['-', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0], ['-', 1.0], ['+', 1.0], ['-', 1.0]]
J [['+', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0]]
humidity 0.09183673469387743
wind
Starting Info:  0.4591836734693877
J [['-', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0]]
J [['-', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0], ['-', 1.0]]
wind 0.030612244897959162
outlook


### Majority Error

In [1065]:
root = ID3(s, rem, full_attr, majorityError, None, stopping=stop, printing=True)

q = queue.Queue()
q2 = queue.Queue()
q.put(root)
childQ = queue.Queue()
childQ.put(True)


unravelTree(q, q2, childQ, 0)

outlook
Starting Info:  0.35714285714285715
J [['-', 1.0], ['-', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0]]
J [['+', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0]]
J [['+', 1.0], ['+', 1.0], ['-', 1.0], ['+', 1.0], ['-', 1.0]]
outlook 0.0714285714285714
temperature
Starting Info:  0.35714285714285715
J [['-', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0]]
J [['+', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0], ['-', 1.0]]
J [['+', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0]]
temperature 5.551115123125783e-17
humidity
Starting Info:  0.35714285714285715
J [['-', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0], ['-', 1.0], ['+', 1.0], ['-', 1.0]]
J [['+', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0]]
humidity 0.07142857142857145
wind
Starting Info:  0.35714285714285715
J [['-', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0]]
J [['-', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0], ['-', 1.0]]
wind 5.551115123125783e-17
hum

#Using modified Tennis Example

In [1066]:
s_tennis_modified = [["sunny", "hot", "high", "weak", "-"],
     ["sunny", "hot", "high", "strong", "-"],
     ["overcast", "hot", "high", "weak", "+"],
     ["rainy", "mild", "high", "weak", "+"],
     ["rainy", "cool", "normal", "weak", "+"],
     ["rainy", "cool", "normal", "strong", "-"],
     ["overcast", "cool", "normal", "strong", "+"],
     ["sunny", "mild", "high", "weak", "-"],
     ["sunny", "cool", "normal", "weak", "+"],
     ["rainy", "mild", "normal", "weak", "+"],
     ["sunny", "mild", "normal", "strong", "+"],
     ["overcast", "mild", "high", "strong", "+"],
     ["overcast", "hot", "normal", "weak", "+"],
     ["rainy", "mild", "high", "strong", "-"],
     #Modifications buddy
     ["overcast", "mild", "normal", "weak", "+"]]

In [1067]:
root = ID3(s_tennis_modified, rem, full_attr, entropy, None, stopping=stop, printing=True)

q = queue.Queue()
q2 = queue.Queue()
q.put(root)
childQ = queue.Queue()
childQ.put(True)


unravelTree(q, q2, childQ, 0)

outlook
Starting Info:  0.9182958340544896
J [['-', 1.0], ['-', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0]]
J [['+', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0]]
J [['+', 1.0], ['+', 1.0], ['-', 1.0], ['+', 1.0], ['-', 1.0]]
outlook 0.27099543775137724
temperature
Starting Info:  0.9182958340544896
J [['-', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0]]
J [['+', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0], ['-', 1.0], ['+', 1.0]]
J [['+', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0]]
temperature 0.032498735534292944
humidity
Starting Info:  0.9182958340544896
J [['-', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0], ['-', 1.0], ['+', 1.0], ['-', 1.0]]
J [['+', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0]]
humidity 0.1686216675320541
wind
Starting Info:  0.9182958340544896
J [['-', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0]]
J [['-', 1.0], ['-', 1.0], ['+', 1.0], ['+', 1.0], ['+', 1.0],

# Used to help with fractional counts

In [1068]:
def entropyWeighted(positive, negative):
  whole = positive + negative
  return (-(positive/whole)*math.log2(positive/whole) if positive/whole > 0 else 0)-((negative/whole)*math.log2(negative/whole) if negative/whole > 0 else 0)

In [1069]:
entropyWeighted(3+5/14, 2)

0.9531971725430561

In [1070]:
poss = [3+5/14, 2+5/14]
negs = [1, 1]
S = 15
sum = 0
for i in range(len(poss)):
  S_v = poss[i] + negs[i]
  print("Pos: ", poss[i], " Neg: ", negs[i], " entropy weighted: ", (S_v)/S*entropyWeighted(poss[i], negs[i]))
  sum += (S_v)/S*entropyWeighted(poss[i], negs[i])
sum

Pos:  3.357142857142857  Neg:  1  entropy weighted:  0.22574444120800313
Pos:  2.357142857142857  Neg:  1  entropy weighted:  0.19665571991054065


0.4224001611185438

# Car Example problem 2

## Set Data

can set stop to halt tree growth after a certain number of steps

In [1071]:
index = 2
s = [s_booltab, s_tennis, s_cartraining, s_banktraining, s_banktraining_filled][index]
s_test = ["","", s_cartesting, s_banktesting, s_banktraining_filled][index]
rem = [remaining_attributes_booltab, remaining_attributes_tennis, remaining_attributes_car, remaining_attributes_bank, remaining_attributes_bank][index]
full_attr = [full_attributes_booltab, full_attributes_tennis, full_attributes_car, full_attributes_bank, full_attributes_bank_filled][index]
pur = [entropy, GiniInd, majorityError][0]
stop = 4

## Running with entropy

In [1072]:
root = ID3(s, rem, full_attr, entropy, None, stopping=stop)

findAcc(root, s_test, full_attr)

0.853021978021978

## Running with gini index

In [1073]:
root = ID3(s, rem, full_attr, GiniInd, None, stopping=stop)

findAcc(root, s_test, full_attr)

0.8667582417582418

## Running with majority error

In [1074]:
root = ID3(s, rem, full_attr, majorityError, None, stopping=stop)

findAcc(root, s_test, full_attr)

0.8571428571428571

## Running through all levels & purity functions

In [1075]:
for purity in range(0, 3):
  print("~~~~~~~~~~~~~~~~~~~~~~~~~")
  print("Using:", ["entropy", "GiniInd", "majorityError"][purity])
  for i in range(1, 7):
    root = ID3(s, rem, full_attr, [entropy, GiniInd, majorityError][purity], None, stopping=i)
    accTest = 1-findAcc(root, s_test, full_attr)
    accTrain = 1-findAcc(root, s, full_attr)
    print("Stopping at", i, "obtains test error", accTest, "and training error", accTrain)

~~~~~~~~~~~~~~~~~~~~~~~~~
Using: entropy
Stopping at 1 obtains test error 0.29670329670329665 and training error 0.30200000000000005
Stopping at 2 obtains test error 0.22252747252747251 and training error 0.22199999999999998
Stopping at 3 obtains test error 0.1964285714285714 and training error 0.18100000000000005
Stopping at 4 obtains test error 0.146978021978022 and training error 0.08199999999999996
Stopping at 5 obtains test error 0.08791208791208793 and training error 0.027000000000000024
Stopping at 6 obtains test error 0.08791208791208793 and training error 0.0
~~~~~~~~~~~~~~~~~~~~~~~~~
Using: GiniInd
Stopping at 1 obtains test error 0.29670329670329665 and training error 0.30200000000000005
Stopping at 2 obtains test error 0.22252747252747251 and training error 0.22199999999999998
Stopping at 3 obtains test error 0.18406593406593408 and training error 0.17600000000000005
Stopping at 4 obtains test error 0.1332417582417582 and training error 0.08899999999999997
Stopping at 5 obt

# Bank Example Problem 3

## Unknowns as a value

In [1076]:
index = 3
s = [s_booltab, s_tennis, s_cartraining, s_banktraining, s_banktraining_filled][index]
s_test = ["","", s_cartesting, s_banktesting, s_banktraining_filled][index]
rem = [remaining_attributes_booltab, remaining_attributes_tennis, remaining_attributes_car, remaining_attributes_bank, remaining_attributes_bank][index]
full_attr = [full_attributes_booltab, full_attributes_tennis, full_attributes_car, full_attributes_bank, full_attributes_bank_filled][index]
pur = [entropy, GiniInd, majorityError][0]
stop = 4

In [1077]:
for purity in range(0, 3):
  print("~~~~~~~~~~~~~~~~~~~~~~~~~")
  print("Using:", ["entropy", "GiniInd", "majorityError"][purity])
  for i in range(1, 17):
    root = ID3(s, rem, full_attr, [entropy, GiniInd, majorityError][purity], None, stopping=i)
    accTest = 1-findAcc(root, s_test, full_attr)
    accTrain = 1-findAcc(root, s, full_attr)
    print("Stopping at", i, "obtains test error", accTest, "and training error", accTrain)

~~~~~~~~~~~~~~~~~~~~~~~~~
Using: entropy
Stopping at 1 obtains test error 0.12480000000000002 and training error 0.11919999999999997


KeyboardInterrupt: 

## Car with unknowns filled

In [ ]:
index = 4
s = [s_booltab, s_tennis, s_cartraining, s_banktraining, s_banktraining_filled][index]
s_test = ["","", s_cartesting, s_banktesting, s_banktesting_filled][index]
rem = [remaining_attributes_booltab, remaining_attributes_tennis, remaining_attributes_car, remaining_attributes_bank, remaining_attributes_bank][index]
full_attr = [full_attributes_booltab, full_attributes_tennis, full_attributes_car, full_attributes_bank, full_attributes_bank_filled][index]
pur = [entropy, GiniInd, majorityError][0]
stop = 4

In [ ]:
for purity in range(0, 3):
  print("~~~~~~~~~~~~~~~~~~~~~~~~~")
  print("Using:", ["entropy", "GiniInd", "majorityError"][purity])
  for i in range(1, 17):
    root = ID3(s, rem, full_attr, [entropy, GiniInd, majorityError][purity], None, stopping=i)
    accTest = 1-findAcc(root, s_test, full_attr)
    accTrain = 1-findAcc(root, s, full_attr)
    print("Stopping at", i, "obtains test error", accTest, "and training error", accTrain)

~~~~~~~~~~~~~~~~~~~~~~~~~
Using: entropy
Stopping at 1 obtains test error 0.12480000000000002 and training error 0.11919999999999997
Stopping at 2 obtains test error 0.11140000000000005 and training error 0.10599999999999998
Stopping at 3 obtains test error 0.10899999999999999 and training error 0.10219999999999996
Stopping at 4 obtains test error 0.11960000000000004 and training error 0.08840000000000003
Stopping at 5 obtains test error 0.13160000000000005 and training error 0.07199999999999995
Stopping at 6 obtains test error 0.13460000000000005 and training error 0.05700000000000005
Stopping at 7 obtains test error 0.1422 and training error 0.04720000000000002
Stopping at 8 obtains test error 0.1462 and training error 0.03400000000000003
Stopping at 9 obtains test error 0.15200000000000002 and training error 0.025399999999999978
Stopping at 10 obtains test error 0.1572 and training error 0.019199999999999995
Stopping at 11 obtains test error 0.15859999999999996 and training error 0.